In [1]:
# install.packages("magrittr")
# install.packages("purrr")
# install.packages("glue")
# install.packages("stringr")
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract



Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [7]:
# The website in html
page = read_html("https://www.audioculture.co.nz/music_index?category=Person")

# Profile names
Profile = page %>% html_nodes(".skippy-col-link") %>% html_text()

# Profile links
Link = page %>% html_nodes(".skippy-col-link") %>% html_attr('href')

# Dataframe of profile and associated link
df = data.frame(Profile, Link)

# Takes a profile link as input
# Returns profile labels (profile members and labels have the same class name, need to seperate)
getLabels = function(x) {
    return(glue('https://www.audioculture.co.nz{x}') %>% read_html() %>% html_nodes("[class='body fact']") %>% html_text())
}

# Takes a profile link as input
# Returns profile's discography with featuring artists as a dataframe
getDiscography = function(x) {
    Discography = glue('https://www.audioculture.co.nz{x}/discography') %>% read_html() %>% html_nodes("[class='header']") %>% html_text()
    Featuring = glue('https://www.audioculture.co.nz{x}/discography') %>% read_html() %>% html_nodes("[class='body']") %>% html_text()
    return(data.frame(Discography, Featuring))
}

# Takes a profile name as input
# Returns given name's discography (currenly no error handling if name not in dataframe)
searchProfile = function(name) {
    name_link = (df %>% filter(str_detect(Profile, name)))[[2]]
    return(getDiscography(name_link))
}

In [9]:
searchProfile('Jamie McDell')


Discography,Featuring
<chr>,<chr>
Six Strings And A Sailboat,Jamie McDell
Ask Me Anything,Jamie McDell
